In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jul 19 12:27:23 2022

@author: b8008458
"""

import pandas as pd
import warnings
import networkx as nx
import geopandas as gpd
import osmnx as ox
from shapely.ops import split
import momepy
from shapely.geometry import Point

#%%


In [2]:

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=PendingDeprecationWarning)
ox.config(use_cache=True, log_console=False)

c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [3]:
#%%

place = "York, United Kingdom"
# get location
cities = ox.geocode_to_gdf([place])

# get all amenities for a given study area

tags = {"amenity": ["cafe","pub","restaurant",
                    "college","kindergarten","library","school","university"
                    ,"bicycle_parking","bicycle_repair_station","bicycle_rental","bus_station","ferry_terminal",
                    "taxi","atm","bank","bureau_de_change",
                    "clinic","dentist","doctors","hospital","pharmacy","social_facility","veterinary"
                    ,"arts_centre","cinema","community_centre","public_bookcase",
                    "social_centre", "theatre", 
                    "police", "fire_station", "post_office", "townhall",
                    "drinking_water","toilets","water_point",
                    "recycling"]}

pois = ox.geometries_from_place(place, tags)



# get some barriers
barriers = gpd.read_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\York\Barriers\YorkBarriers.shp")


# get population weighted centroids
pwc = gpd.read_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\York\Centriods\YorkLSOAsSinglePartGPKG.gpkg")


In [4]:
# pre-process points of interest
pois = pois.reset_index(drop=True).explode().reset_index(drop=True) # avoid multipart pois
pois['poiID'] = range(0,len(pois))
pois = pois.set_index('poiID')

# convert polygons to points
pois['geometry'] = pois.centroid

# clean columns
pois = pois[['geometry','amenity']]


C:\Users\b8008458\AppData\Local\Temp\ipykernel_8704\1491351608.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  pois = pois.reset_index(drop=True).explode().reset_index(drop=True) # avoid multipart pois
C:\Users\b8008458\AppData\Local\Temp\ipykernel_8704\1491351608.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pois['geometry'] = pois.centroid


In [ ]:

# function to get isochrones
def get_isochrone(
    lon, lat, walk_times=[15, 30], speed=4.5, name=None, point_index=None
):
    loc = (lat, lon)
    G = ox.graph_from_point(loc, simplify=True, network_type="all")
    gdf_nodes = ox.graph_to_gdfs(G, edges=False)
    center_node = ox.distance.nearest_nodes(G, lon, lat)

    meters_per_minute = speed * 1000 / 60  # km per hour to m per minute
    for u, v, k, data in G.edges(data=True, keys=True):
        data["time"] = data["length"] / meters_per_minute
    polys = []
    for walk_time in walk_times:
        subgraph = nx.ego_graph(G, center_node, radius=walk_time, distance="time")
        node_points = [
            Point(data["x"], data["y"]) for node, data in subgraph.nodes(data=True)
        ]
        polys.append(gpd.GeoSeries(node_points).unary_union.convex_hull)
    info = {}
    if name:
        info["name"] = [name for t in walk_times]
    if point_index:
        info["point_index"] = [point_index for t in walk_times]
    return {**{"geometry": polys, "time": walk_times}, **info}



In [ ]:

#%%

WT = [5, 10, 15]
BARRIERS = len(barriers)

# build geopandas data frame of isochrone polygons for each barrier
isochrones = pd.concat(
    [
        gpd.GeoDataFrame(
            get_isochrone(
                r["geometry"].x,
                r["geometry"].y,
                name=r["globalid"],
                point_index=i,
                walk_times=WT,
            ),
            crs=barriers.crs,
        )
        for i, r in barriers.head(BARRIERS).iterrows()
    ]
)



In [ ]:

#%%

warnings.filterwarnings("ignore")

gdf = isochrones.set_index(["time", "point_index"]).copy()
# remove shorter walk time from longer walk time polygon to make folium work better
for idx in range(len(WT)-1,0,-1):
    gdf.loc[WT[idx], "geometry"] = (
        gdf.loc[WT[idx]]
        .apply(
            lambda r: r["geometry"].symmetric_difference(
                gdf.loc[(WT[idx-1], r.name), "geometry"]
            ),
            axis=1,
        )
        .values
    )

m = gdf.reset_index().explore(column="time", height=300, width=500, scheme="boxplot")
barriers.head(BARRIERS).explore(m=m, marker_kwds={"radius": 3, "color": "red"})


With Isochrones calcualted, find PWC within 15 minutes of barrier

In [ ]:
# remove shorter walk distances
isochrones = isochrones[isochrones.time == 15]
isochrones['point_index'] = isochrones['point_index'].fillna(0)
isochrones.head(10)

In [ ]:
join_inner_df = pois.sjoin(isochrones, how='inner')

In [ ]:
result = join_inner_df.groupby('name').size()

Now split the network at barrier locations to prevent routes to pass through

In [8]:

# function to get isochrones
def get_isochrone(
    lon, lat, walk_times=[15, 30], speed=4.5, name=None, point_index=None
):
    loc = (lat, lon)
    G = ox.graph_from_point(loc, simplify=True, network_type="all")
    # disconnect network at barriers
    G_edges = ox.graph_to_gdfs(ox.get_undirected(G), nodes=False, edges=True, node_geometry=False, fill_edge_geometry=True)
    ####
    shply_line = G_edges.geometry.unary_union
    point = barriers.to_crs(G_edges.crs)
    for i in range(len(point)):
         print(shply_line.interpolate(shply_line.project( point.geometry[i])).wkt)
    result = point.copy()
    result['geometry'] = result.apply(lambda row: shply_line.interpolate(shply_line.project( row.geometry)), axis=1)
    buffer = result.geometry.buffer(0.00001)
    broken_network = G_edges.intersects(buffer.unary_union)
    broken_network = pd.DataFrame(broken_network)
    broken_network.rename(columns = {0:'Clipped'}, inplace = True)
    broken_network = pd.concat([broken_network,G_edges],axis=1)

    # drop intersected lines
    broken_network.drop(broken_network[broken_network['Clipped'] == True].index, inplace=True)
    broken_network = broken_network[['geometry','to','from']]
    broken_network = gpd.GeoDataFrame(broken_network, geometry='geometry')
    G_edges = broken_network
    G = momepy.gdf_to_nx(G_edges)
    gdf_nodes = ox.graph_to_gdfs(G, edges=False)
    center_node = ox.distance.nearest_nodes(G, lon, lat)

    meters_per_minute = speed * 1000 / 60  # km per hour to m per minute
    for u, v, k, data in G.edges(data=True, keys=True):
        data["time"] = data["length"] / meters_per_minute
    polys = []
    for walk_time in walk_times:
        subgraph = nx.ego_graph(G, center_node, radius=walk_time, distance="time")
        node_points = [
            Point(data["x"], data["y"]) for node, data in subgraph.nodes(data=True)
        ]
        polys.append(gpd.GeoSeries(node_points).unary_union.convex_hull)
    info = {}
    if name:
        info["name"] = [name for t in walk_times]
    if point_index:
        info["point_index"] = [point_index for t in walk_times]
    return {**{"geometry": polys, "time": walk_times}, **info}

In [9]:
#%%

WT = [5, 10, 15]
BARRIERS = len(barriers)

# build geopandas data frame of isochrone polygons for each barrier
isochrones = pd.concat(
    [
        gpd.GeoDataFrame(
            get_isochrone(
                r["geometry"].x,
                r["geometry"].y,
                name=r["globalid"],
                point_index=i,
                walk_times=WT,
            ),
            crs=barriers.crs,
        )
        for i, r in barriers.head(BARRIERS).iterrows()
    ]
)



POINT (-1.0830741947186202 53.94545736287081)
POINT (-1.0814849565149334 53.94450330968466)
POINT (-1.0806276753422634 53.94422234487637)
POINT (-1.0807137621110599 53.94503403314828)
POINT (-1.0798159007576296 53.945060618454185)
POINT (-1.0733718 53.945276)
POINT (-1.0712696 53.9469214)
POINT (-1.0894078 53.9527534)
POINT (-1.0894078 53.9527534)
POINT (-1.0679221 53.9484921)
POINT (-1.0679221 53.9484921)
POINT (-1.0678573256480344 53.94771482221234)
POINT (-1.067911460985559 53.948363639115826)
POINT (-1.0678988921289714 53.94821187633374)
POINT (-1.067837 53.9531551)
POINT (-1.0685453 53.9532207)
POINT (-1.0685453 53.9532207)
POINT (-1.0685453 53.9532207)
POINT (-1.067837 53.9531551)
POINT (-1.067837 53.9531551)
POINT (-1.0685453 53.9532207)
POINT (-1.0685453 53.9532207)
POINT (-1.0685453 53.9532207)
POINT (-1.0685453 53.9532207)
POINT (-1.0685453 53.9532207)
POINT (-1.0725484085617611 53.95411006661373)
POINT (-1.0752561 53.9543577)
POINT (-1.0752561 53.9543577)
POINT (-1.0752561 5

c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\Users\b8008458\AppData\Local\Temp\ipykernel_8704\215476077.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buffer = result.geometry.buffer(0.00001)
c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\momepy\utils.py:142: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


KeyError: 'x'

In [7]:
G = ox.graph_from_place('York, United Kingdom', network_type="all",clean_periphery=True, simplify=True)
G_edges = ox.graph_to_gdfs(ox.get_undirected(G), nodes=False, edges=True, node_geometry=False, fill_edge_geometry=True)
shply_line = G_edges.geometry.unary_union
point = barriers.to_crs(G_edges.crs)
for i in range(len(point)):
    print(shply_line.interpolate(shply_line.project( point.geometry[i])).wkt)
result = point.copy()
result['geometry'] = result.apply(lambda row: shply_line.interpolate(shply_line.project( row.geometry)), axis=1)
buffer = result.geometry.buffer(0.00001)
broken_network = G_edges.intersects(buffer.unary_union)
broken_network = pd.DataFrame(broken_network)
broken_network.rename(columns = {0:'Clipped'}, inplace = True)
broken_network = pd.concat([broken_network,G_edges],axis=1)

# drop intersected lines
broken_network.drop(broken_network[broken_network['Clipped'] == True].index, inplace=True)
broken_network = broken_network[['geometry','to','from']]
broken_network = gpd.GeoDataFrame(broken_network, geometry='geometry')
G_edges = broken_network.to_crs(3857)
G_edges['length'] = G_edges.length
G = momepy.gdf_to_nx(G_edges)
    

POINT (-1.0830741947186198 53.94545736287081)
POINT (-1.0814849565149336 53.94450330968466)
POINT (-1.0806276753422634 53.94422234487637)
POINT (-1.08071376211106 53.94503403314828)
POINT (-1.0798159007576293 53.945060618454185)
POINT (-1.0729077482000908 53.94522170317522)
POINT (-1.070175884167186 53.94533768694834)
POINT (-1.066155850653157 53.946130392666426)
POINT (-1.0596793918931255 53.94750844774118)
POINT (-1.0584621374557899 53.9498835122274)
POINT (-1.053504493096702 53.94999378875074)
POINT (-1.0483957 53.9493786)
POINT (-1.0471864188190294 53.95007192153852)
POINT (-1.044838912161834 53.95011707986839)
POINT (-1.0457341148261812 53.95355378085258)
POINT (-1.0507618550857991 53.95965211867487)
POINT (-1.0488622915195192 53.96004194072127)
POINT (-1.0467787672431732 53.96014118257044)
POINT (-1.0435420391541559 53.96062926684852)
POINT (-1.0383891703797412 53.96130333987923)
POINT (-1.0573538914830212 53.95972082289049)
POINT (-1.0569303 53.959484)
POINT (-1.0603111540514172

c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\Users\b8008458\AppData\Local\Temp\ipykernel_8704\4011565485.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buffer = result.geometry.buffer(0.00001)
c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\momepy\utils.py:142: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [12]:
G_edges.crs
test = G_edges.to_crs(3857)
test.crs
test['length'] = test.length


In [13]:
test.head(10)

geometry  \
u       v          key                                                      
3470523 9230022776 0    LINESTRING (-113934.675 7168192.149, -113945.7...   
        3470558    0    LINESTRING (-113825.682 7168242.135, -113836.7...   
        27189468   0    LINESTRING (-113912.578 7168089.073, -113913.9...   
3470533 2669055381 0    LINESTRING (-113983.478 7168334.421, -113974.2...   
        3470543    0    LINESTRING (-113983.478 7168334.421, -113965.9...   
        2669055376 0    LINESTRING (-113999.396 7168296.590, -113994.6...   
3470543 3470558    0    LINESTRING (-113871.557 7168370.302, -113849.6...   
        5750402895 0    LINESTRING (-113735.191 7169163.698, -113763.6...   
3470558 4860580616 0    LINESTRING (-113825.682 7168242.135, -113837.1...   
3471820 4161711206 0    LINESTRING (-112761.301 7170504.397, -112795.5...   

                                to        from       length  
u       v          key                                       
3470523 9230022776 0    9230022776     3470523    11.546218  
        3470558    0       3470523     3470558   128.207959  
        27189468   0       3470523    27189468   108.436021  
3470533 2669055381 0    2669055381     3470533    25.780170  
        3470543    0       3470543     3470533   125.854428  
        2669055376 0       3470533  2669055376    41.535236  
3470543 3470558    0       3470558     3470543   150.360308  
        5750402895 0       3470543  5750402895   869.063788  
3470558 4860580616 0    4860580616     3470558  2923.874064  
3471820 4161711206 0       3471820  4161711206    57.217087